# TPOT tutorial on the Titanic dataset

The Titanic machine learning competition on [Kaggle](https://www.kaggle.com/c/titanic) is one of the most popular beginner's competitions on the platform. We will use that competition here to demonstrate the implementation of TPOT.

In [3]:
# Import required libraries
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pandas as pd 
import numpy as np

In [4]:
# Load the data
titanic = pd.read_csv('/dbfs/titanic/train.csv')
titanic.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Data Exploration

In [6]:
titanic.groupby('Sex').Survived.value_counts()

Out[3]: Sex Survived
female 1 233
 0 81
male 0 468
 1 109
Name: Survived, dtype: int64

In [7]:
titanic.groupby(['Pclass','Sex']).Survived.value_counts()

Out[4]: Pclass Sex Survived
1 female 1 91
 0 3
 male 0 77
 1 45
2 female 1 70
 0 6
 male 0 91
 1 17
3 female 0 72
 1 72
 male 0 300
 1 47
Name: Survived, dtype: int64

In [8]:
id = pd.crosstab([titanic.Pclass, titanic.Sex], titanic.Survived.astype(float))
id.div(id.sum(1).astype(float), 0)

Survived 
 0.0 
 1.0 
 
 
 Pclass 
 Sex 
 
 
 
 
 
 
 1 
 female 
 0.031915 
 0.968085 
 
 
 male 
 0.631148 
 0.368852 
 
 
 2 
 female 
 0.078947 
 0.921053 
 
 
 male 
 0.842593 
 0.157407 
 
 
 3 
 female 
 0.500000 
 0.500000 
 
 
 male 
 0.864553 
 0.135447

## Data Munging

The first and most important step in using TPOT on any data set is to rename the target class/response variable to `class`.

In [11]:
titanic.rename(columns={'Survived': 'class'}, inplace=True)

At present, TPOT requires all the data to be in numerical format. As we can see below, our data set has 5 categorical variables which contain non-numerical values: `Name`, `Sex`, `Ticket`, `Cabin` and `Embarked`.

In [13]:
titanic.dtypes

Out[7]: PassengerId int64
class int64
Pclass int64
Name object
Sex object
Age float64
SibSp int64
Parch int64
Ticket object
Fare float64
Cabin object
Embarked object
dtype: object

We then check the number of levels that each of the five categorical variables have.

In [15]:
for cat in ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']:
    print("Number of levels in category '{0}': \b {1:2.2f} ".format(cat, titanic[cat].unique().size))

Number of levels in category 'Name':  891.00 
Number of levels in category 'Sex':  2.00 
Number of levels in category 'Ticket':  681.00 
Number of levels in category 'Cabin':  148.00 
Number of levels in category 'Embarked':  4.00

As we can see, `Sex` and `Embarked` have few levels. Let's find out what they are.

In [17]:
for cat in ['Sex', 'Embarked']:
    print("Levels for catgeory '{0}': {1}".format(cat, titanic[cat].unique()))

Levels for catgeory 'Sex': ['male' 'female']
Levels for catgeory 'Embarked': ['S' 'C' 'Q' nan]

We then code these levels manually into numerical values. For `nan` i.e. the missing values, we simply replace them with a placeholder value (-999). In fact, we perform this replacement for the entire data set.

In [19]:
titanic['Sex'] = titanic['Sex'].map({'male':0,'female':1})
titanic['Embarked'] = titanic['Embarked'].map({'S':0,'C':1,'Q':2})

In [20]:
titanic = titanic.fillna(-999)
pd.isnull(titanic).any()

Out[11]: PassengerId False
class False
Pclass False
Name False
Sex False
Age False
SibSp False
Parch False
Ticket False
Fare False
Cabin False
Embarked False
dtype: bool

Since `Name` and `Ticket` have so many levels, we drop them from our analysis for the sake of simplicity. For `Cabin`, we encode the levels as digits using Scikit-learn's [`MultiLabelBinarizer`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html) and treat them as new features.

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
CabinTrans = mlb.fit_transform([{str(val)} for val in titanic['Cabin'].values])

In [23]:
CabinTrans

Out[13]: array([[1, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [1, 0, 0, ..., 0, 0, 0],
 ...,
 [1, 0, 0, ..., 0, 0, 0],
 [0, 0, 0, ..., 0, 0, 0],
 [1, 0, 0, ..., 0, 0, 0]])

Drop the unused features from the dataset.

In [25]:
titanic_new = titanic.drop(['Name','Ticket','Cabin','class'], axis=1)

In [26]:
assert (len(titanic['Cabin'].unique()) == len(mlb.classes_)), "Not Equal" #check correct encoding done

We then add the encoded features to form the final dataset to be used with TPOT.

In [28]:
titanic_new = np.hstack((titanic_new.values,CabinTrans))

In [29]:
np.isnan(titanic_new).any()

Out[17]: False

Keeping in mind that the final dataset is in the form of a numpy array, we can check the number of features in the final dataset as follows.

In [31]:
titanic_new[0].size

Out[18]: 156

Finally we store the class labels, which we need to predict, in a separate variable.

In [33]:
titanic_class = titanic['class'].values

## Data Analysis using TPOT

To begin our analysis, we need to divide our training data into training and validation sets. The validation set is just to give us an idea of the test set error. The model selection and tuning is entirely taken care of by TPOT, so if we want to, we can skip creating this validation set.

In [36]:
training_indices, validation_indices = training_indices, testing_indices = train_test_split(titanic.index, stratify = titanic_class, train_size=0.75, test_size=0.25)
training_indices.size, validation_indices.size

Out[20]: (668, 223)

After that, we proceed to calling the `fit`, `score` and `export` functions on our training dataset. To get a better idea of how these functions work, refer the TPOT documentation [here](http://epistasislab.github.io/tpot/api/).

An important TPOT parameter to set is the number of generations. Since our aim is to just illustrate the use of TPOT, we have set maximum optimization time to 2 minutes (max_time_mins=2). On a standard laptop with 4GB RAM, it roughly takes 5 minutes per generation to run. For each added generation, it should take 5 mins more. Thus, for the default value of 100, total run time could be roughly around 8 hours.

In [38]:
tpot = TPOTClassifier(verbosity=2, max_time_mins=2, max_eval_time_mins=0.04, population_size=40)
tpot.fit(titanic_new[training_indices], titanic_class[training_indices])

Optimization Progress: 0%| | 0/40 [00:00<?, ?pipeline/s]Optimization Progress: 10%|█ | 4/40 [00:01<00:09, 3.82pipeline/s]Optimization Progress: 15%|█▌ | 6/40 [00:01<00:07, 4.85pipeline/s]Optimization Progress: 20%|██ | 8/40 [00:01<00:07, 4.57pipeline/s]Optimization Progress: 25%|██▌ | 10/40 [00:02<00:08, 3.40pipeline/s]Optimization Progress: 28%|██▊ | 11/40 [00:04<00:18, 1.60pipeline/s]Optimization Progress: 38%|███▊ | 15/40 [00:05<00:13, 1.91pipeline/s]Optimization Progress: 42%|████▎ | 17/40 [00:05<00:10, 2.25pipeline/s]Optimization Progress: 48%|████▊ | 19/40 [00:07<00:10, 1.91pipeline/s]Optimization Progress: 50%|█████ | 20/40 [00:07<00:08, 2.33pipeline/s]Optimization Progress: 52%|█████▎ | 21/40 [00:08<00:11, 1.65pipeline/s]Optimization Progress: 55%|█████▌ | 22/40 [00:08<00:11, 1.60pipeline/s]Optimization Progress: 60%|██████ | 24/40 [00:09<00:07, 2.16pipeline/s]Optimization Progress: 68%|██████▊ | 27/40 [00:09<00:05, 2.50pipeline/s]Optimization Progress: 70%|███████ | 28/40 [00:10<00:04, 2.89pipeline/s]Optimization Progress: 72%|███████▎ | 29/40 [00:10<00:05, 2.09pipeline/s]Optimization Progress: 75%|███████▌ | 30/40 [00:11<00:03, 2.68pipeline/s]Optimization Progress: 78%|███████▊ | 31/40 [00:11<00:03, 2.44pipeline/s]Optimization Progress: 80%|████████ | 32/40 [00:12<00:03, 2.10pipeline/s]Optimization Progress: 82%|████████▎ | 33/40 [00:12<00:02, 2.48pipeline/s]Optimization Progress: 88%|████████▊ | 35/40 [00:14<00:03, 1.62pipeline/s]Optimization Progress: 95%|█████████▌| 38/40 [00:15<00:00, 2.10pipeline/s]Optimization Progress: 98%|█████████▊| 39/40 [00:15<00:00, 1.83pipeline/s]Optimization Progress: 100%|██████████| 40/40 [00:15<00:00, 2.35pipeline/s]Optimization Progress: 41pipeline [00:17, 1.36pipeline/s] Optimization Progress: 52%|█████▎ | 42/80 [00:19<00:42, 1.12s/pipeline]Optimization Progress: 54%|█████▍ | 43/80 [00:19<00:30, 1.20pipeline/s]Optimization Progress: 57%|█████▊ | 46/80 [00:20<00:24, 1.38pipeline/s]Optimization Progress: 59%|█████▉ | 47/80 [00:21<00:18, 1.83pipeline/s]Optimization Progress: 61%|██████▏ | 49/80 [00:21<00:13, 2.36pipeline/s]Optimization Progress: 62%|██████▎ | 50/80 [00:22<00:19, 1.55pipeline/s]Optimization Progress: 64%|██████▍ | 51/80 [00:22<00:14, 2.00pipeline/s]Optimization Progress: 66%|██████▋ | 53/80 [00:23<00:11, 2.28pipeline/s]Optimization Progress: 68%|██████▊ | 54/80 [00:23<00:09, 2.71pipeline/s]Optimization Progress: 70%|███████ | 56/80 [00:23<00:06, 3.47pipeline/s]Optimization Progress: 71%|███████▏ | 57/80 [00:25<00:18, 1.24pipeline/s]Optimization Progress: 76%|███████▋ | 61/80 [00:26<00:12, 1.57pipeline/s]Optimization Progress: 78%|███████▊ | 62/80 [00:26<00:08, 2.04pipeline/s]Optimization Progress: 79%|███████▉ | 63/80 [00:27<00:09, 1.72pipeline/s]Optimization Progress: 81%|████████▏ | 65/80 [00:27<00:06, 2.21pipeline/s]Optimization Progress: 84%|████████▍ | 67/80 [00:28<00:04, 3.01pipeline/s]Optimization Progress: 85%|████████▌ | 68/80 [00:28<00:03, 3.37pipeline/s]Optimization Progress: 86%|████████▋ | 69/80 [00:28<00:04, 2.56pipeline/s]Optimization Progress: 88%|████████▊ | 70/80 [00:29<00:03, 3.24pipeline/s]Optimization Progress: 89%|████████▉ | 71/80 [00:29<00:02, 3.66pipeline/s]Optimization Progress: 91%|█████████▏| 73/80 [00:29<00:01, 3.67pipeline/s]Optimization Progress: 92%|█████████▎| 74/80 [00:31<00:03, 1.53pipeline/s]Optimization Progress: 94%|█████████▍| 75/80 [00:32<00:03, 1.46pipeline/s]Optimization Progress: 96%|█████████▋| 77/80 [00:34<00:02, 1.28pipeline/s]Optimization Progress: 98%|█████████▊| 78/80 [00:34<00:01, 1.64pipeline/s]Optimization Progress: 81pipeline [00:35, 1.96pipeline/s] Optimization Progress: 82pipeline [00:35, 1.68pipeline/s] Generation 1 - Current best internal CV score: 0.821849399618449
Optimization Progress: 82pipeline [00:35, 1.68pipeline/s]Optimization Progress: 69%|██████▉ | 83/120 [00:38<00:39, 1.06s/pipeline]Optimization Progress: 70%|███████ | 84/120 [00:38<00:31, 1.16pip

In [39]:
tpot.score(titanic_new[validation_indices], titanic.loc[validation_indices, 'class'].values)

Out[22]: 0.8026905829596412

In [40]:
tpot.export('tpot_titanic_pipeline.py')

Let's have a look at the generated code. As we can see, the random forest classifier performed the best on the given dataset out of all the other models that TPOT currently evaluates on. If we ran TPOT for more generations, then the score should improve further.

### Make predictions on the submission data

In [43]:
# Read in the submission dataset
titanic_sub = pd.read_csv('/dbfs/titanic/test.csv')
titanic_sub.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


The most important step here is to check for new levels in the categorical variables of the submission dataset that are absent in the training set. We identify them and set them to our placeholder value of '-999', i.e., we treat them as missing values. This ensures training consistency, as otherwise the model does not know what to do with the new levels in  the submission dataset.

In [45]:
for var in ['Cabin']: #,'Name','Ticket']:
    new = list(set(titanic_sub[var]) - set(titanic[var]))
    titanic_sub.loc[titanic_sub[var].isin(new), var] = -999

We then carry out the data munging steps as done earlier for the training dataset.

In [47]:
titanic_sub['Sex'] = titanic_sub['Sex'].map({'male':0,'female':1})
titanic_sub['Embarked'] = titanic_sub['Embarked'].map({'S':0,'C':1,'Q':2})

In [48]:
titanic_sub = titanic_sub.fillna(-999)
pd.isnull(titanic_sub).any()

Out[31]: PassengerId False
Pclass False
Name False
Sex False
Age False
SibSp False
Parch False
Ticket False
Fare False
Cabin False
Embarked False
dtype: bool

While calling `MultiLabelBinarizer` for the submission data set, we first fit on the training set again to learn the levels and then transform the submission dataset values. This further ensures that only those levels that were present in the training dataset are transformed. If new levels are still found in the submission dataset then it will return an error and we need to go back and check our earlier step of replacing new levels with the placeholder value.

In [50]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
SubCabinTrans = mlb.fit([{str(val)} for val in titanic['Cabin'].values]).transform([{str(val)} for val in titanic_sub['Cabin'].values])
titanic_sub = titanic_sub.drop(['Name','Ticket','Cabin'], axis=1)

In [51]:
# Form the new submission data set
titanic_sub_new = np.hstack((titanic_sub.values,SubCabinTrans))

In [52]:
np.any(np.isnan(titanic_sub_new))

Out[34]: False

In [53]:
# Ensure equal number of features in both the final training and submission dataset
assert (titanic_new.shape[1] == titanic_sub_new.shape[1]), "Not Equal" 

In [54]:
# Generate the predictions
submission = tpot.predict(titanic_sub_new)

In [55]:
# Create the submission file
final = pd.DataFrame({'PassengerId': titanic_sub['PassengerId'], 'Survived': submission})
final.to_csv('/dbfs/titanic/results-tpot.csv', index = False)

In [56]:
final.shape

Out[38]: (418, 2)

There we go! We have successfully generated the predictions for the 418 data points in the submission dataset, and we're good to go ahead to submit these predictions on Kaggle.